# Задание 1

Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.

In [2]:
import pandas as pd

In [3]:
rate = pd.read_csv('ratings.csv')
rate.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [5]:
# Определим пользователей, которые выставили больше всех оценок 
users = rate.groupby('userId').count()
users = users[users['rating'] > 100]
# можно проще через функцию 
users = rate.groupby('userId', as_index = False)['userId'].agg({'qty': 'count'}).query('qty > 100').head(10)
users.head()

,userId,qty
3,4,204
7,8,116
14,15,1700
16,17,363
18,19,423


In [7]:
# Пользователи, выставившие больше 100 оценок c рассчитанными максимальным и минимальным timestamp пользователя
lifetime = users.merge(rate, on = 'userId', how = 'inner').groupby('userId', as_index = False).agg({'timestamp':[max, min]})
lifetime.head(5)

userId   timestamp            
                 max         min
0      4   949982274   949778714
1      8  1154474527  1154389340
2     15  1469330735   997937239
3     17  1127476640  1127468587
4     19   855195373   855190091

In [8]:
(lifetime['timestamp']['max'] - lifetime['timestamp']['min']).mean()

62761416.7

# Задание 2

Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). Необходимо сформировать две таблицы:
таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
аналогичную таблицу по типам выручки с указанием адреса клиента
Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

In [10]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)

In [11]:

auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)

In [12]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)

In [13]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)

In [14]:
revenue = (rzd
            .merge(auto, how='outer')
            .merge(air, how='outer')
            .merge(client_base, how='outer')
       )
revenue.head()

,client_id,rzd_revenue,auto_revenue,air_revenue,address
0,111,1093.0,NaN,NaN,Комсомольская 4
1,112,2810.0,NaN,NaN,Энтузиастов 8а
2,113,10283.0,57483.0,NaN,Левобережная 1а
3,114,5774.0,83.0,NaN,Мира 14
4,115,981.0,912.0,81.0,ЗЖБИиДК 1


In [15]:
# три типа выручки без адресов
revenue_full = revenue.drop(['address'], axis=1)
revenue_full.head()

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,NaN,NaN
1,112,2810.0,NaN,NaN
2,113,10283.0,57483.0,NaN
3,114,5774.0,83.0,NaN
4,115,981.0,912.0,81.0


# Задание 3

В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. Т. е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы. Как бы вы добавили эти сведения в таблицу визитов и покупок? Для составления ответа можно использовать вопросы:
У каждого пользователя известен набор координат. А для связывания с визитом или фактом покупки скорее всего потребуется одно-два числа. Как их получить?
Наборы координат одного и того же пользователя могут быть значительно удалены друг от друга. Как это отразится на вопросе расчетах пункта 1?
Какие дополнительные признаки можно получить из координат? Ведь это просто числа, которые сами по себе мало что дают.


Вариант 1: Выбрать среднюю координату и по ней расчитывать все географические атрибуты (в идеале учитывать и временные интервалы между измерениями местоположения) 

Вариант 2: Отобрать наиболее часто встречающиеся для пользователя гео атрибуты (до заданной точности определять по кординатам: страну, регион, район, город...). Использовать выбранный достоверный атрибут(ы) в качестве сведений о пользователе (например, точно определенный регион)

Вариант 3: Подтягивать информацию с актуальной координаты (последняя по времени измерения)

Варинат 4: Комбинация методов. Выбор метода происходит в зависимости от кол-ва уникальных координат относительно общего числа записей, удаленности координат друг от друга, временного интервала измерений...